In [ ]:
import pandas as pd
import numpy as np
import csv
import re
from bs4 import BeautifulSoup
import camelot 

from tabula import read_pdf
from tabulate import tabulate
from itertools import chain
import matplotlib as plt

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# %matplotlib inline
%config InlineBackend.figure_format='retina'
# If encounter SSL certify error - go to your Python 3.x.x folder and install Certification 

In [ ]:
csv = pd.read_csv('farm_smoke.csv')
sonoma_data = csv.query('County == "Sonoma" and Year >= 2001').copy().reset_index(drop=True)
sonoma_data

In [ ]:
urls = sonoma_data['Links']
urls

In [ ]:
# Note 1: 2021 - 2017 can work together 
# Note 2: 2016 - format diff, have -- and ' .' formating, top 12 reds 
# Note 3: 2015, 2014 - top 12 reds
# Note 4: 2013, 2012 - format change, words and style is diff; need to change camelot

table_list = []
for link, year in zip(urls, sonoma_data['Year']):
    page = '12'
    if year == 2017 or year == 2004 or year == 2003 or year == 2001:
        page = '10'
    elif year == 2007:
        page = '9'
    elif year == 2016 or year == 2013 or year == 2011 or year == 2010 or year == 2009:
        page = '8'
    elif year == 2014 or year == 2012 or year == 2006:
        page = '7'
    elif year == 2008 or year == 2005:
        page = '11'
    # utilize camelot to read pdf file at page 12, 
    # capture precisely with stream and close up the row gap with row_tol = 10

    if year <= 2011:
        if year == 2011:
            # specific reads for 2011
            tables = camelot.read_pdf(link, pages=page, table_areas=['30,720,600,250'], flavor='stream', row_tol=8)[0].df
        elif year == 2004:
            tables = camelot.read_pdf(link, pages=page, table_areas=['60,740,600,370'], flavor='stream', row_tol=8,split_text=True)[0].df
        elif year == 2005:
            tables = camelot.read_pdf(link, pages=page, table_areas=['50,720,600,300'], flavor='stream', row_tol=8, split_text=True)[0].df
        elif year <= 2010 and year != 2005 and year != 2004:
            tables = camelot.read_pdf(link, pages=page, flavor='stream', row_tol=8, split_text=True)[0].df
    elif year != 2012 and year != 2013:
        tables = camelot.read_pdf(link, pages=page, flavor='stream', row_tol=10)[0].df
    
    display(tables)
    table_list.append(tables)
print('finish reading pdf files')


In [ ]:
def rename_replace_varietal_column(table, year):

    if year == 2016:
        table.columns = ['0', '1', '2', '3', '4', '5', '6', '7']
        # For 2016 crop report:
        # temp is non-bearing
        table['Temp'] = table['3']  # Create a temporary column and store 'Bearing' values
        table['3'] = table['2']  # Move 'Non-Bearing' values to 'Bearing' column
        table['2'] = table['Temp']  # Move temporary values to 'Non-Bearing' column
        table.drop('Temp', axis=1, inplace=True)  # Drop the temporary column
        
    elif year <= 2010:
        if year != 2006 and year != 2004 and year != 2003 and year != 2001:
            table.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
            table['7'] = table['6'] + table['7']
            table['9'] = table['8'] + table['9']
            table = table.drop(['6', '8'], axis=1)
        elif year == 2006:
            table.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8']
            table['7'] = table['6'] + table['7']
            table = table.drop(['6'], axis=1)

    new_columns = ['Varietal', 'Year', 'Bearing' , 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
    data = table.rename(columns=dict(zip(table.columns,new_columns)))

    if year == 2011:
        # 2011 specific fixes
        # 24		2011	5,216.8	81.1	5,297.9	13,101.0	$ 2,357.61	$\n30,887,100
        # 25	Zinfandel
        data.loc[24][0] = data.loc[25][0]
        data.loc[27][0] = data.loc[28][0]
        data.loc[28][0] = ''

    elif year <= 2010 and year != 2007 and year != 2005:
        # for 2010; similar for 2009
        # 28		2010	37,804.4	1,606.3	39,410.7	108,481.0\n(a	) $2,255.85 (a	) $244,717,100
        # 29	TOTAL REDS	2009	37,937.2	3,636.2	41,573.4	120,717.0\n(a	) $2,430.05 (a	) $293,348,400	
        # 	
        if year == 2003 or year == 2002 or year == 2001:
            # specific for 2003, 2002, 2001: drop the total all wine grapes row
            data = data[:28]
        data.loc[28][0] = data.loc[29][0]
        data.loc[29][0] = ''
        
    elif year == 2007:
        data.loc[29][0] = data.loc[30][0]
        data.loc[30][0] = ''

    # data drop empty row
    data = data.query("Year != '' and Bearing != '' and Total != '' and Tons != ''")

    data['Varietal'] = data['Varietal'].replace('Mataro/', 'Mataro/Mouvedere')
    data['Varietal'] = data['Varietal'].replace('Cabernet', 'Cabernet Sauvignon')

    # for 2016 only; ignored when dataset does not contain these strings
     # for 2016; Sangiovese/Sangiove to Sangiovese 
    data['Varietal'] = data['Varietal'].replace('Sangiovese/', 'Sangiovese')
    # for 2016 Syrah - Shiraz to Syrah-Shiraz ; 
    data['Varietal'] = data['Varietal'].replace('Syrah - Shiraz', 'Syrah-Shiraz')

    # <= 2010; TOTAL REDS does not include Other Reds; 
    # > 2010; TOTAL ALL REDS include Other Reds
    # 2012, 2011 Only has TOTAL REDS 
    # Starting from <= 2003, Total Reds with Other Reds

    # data['Varietal'] = data['Varietal'].replace('TOTAL ALL REDS', 'TOTAL ALL REDS ')

    if year <= 2010:
        data['Varietal'] = data['Varietal'].replace('Total Reds', 'TOTAL ALL REDS NOT including Other Reds')
        data['Varietal'] = data['Varietal'].replace('TOTAL REDS', 'TOTAL ALL REDS NOT including Other Reds')

    # For x<=2016 and x>2010; total count of all reds is label as TOTAL REDS
    data['Varietal'] = data['Varietal'].replace('TOTAL REDS', 'TOTAL ALL REDS Includes Other Reds')

    # For >= 2017, total count of all reds is label as TOTAL ALL REDS
    data['Varietal'] = data['Varietal'].replace('TOTAL ALL REDS', 'TOTAL ALL REDS Includes Other Reds')

    # for 2005 and 2004 
    data['Varietal'] = data['Varietal'].replace('Cabernet Sauv.', 'Cabernet Sauvigno')
    # for 2004
    data['Varietal'] = data['Varietal'].replace('Syrah-shiraz', 'Syrah-Shiraz')

    # added Mourvedere (a), Sangioveto bc of 2016
    data['Varietal'] = data['Varietal'].replace(['', 'including other reds', 'Mouvedere', 'Mourvedere (a)', 'Sauvignon', 'Sangioveto'], np.nan)
    data = data.reset_index(drop=True)

    # duplicate each variable twice because there is empty space after each variable 
    # For example: Zinfandel 2015 2014 to Zinfandel 2015 Zinfandel 2014
    data['Varietal'] = data['Varietal'].dropna().repeat(2).reset_index(drop=True)

    return data

def add_shift_county_color_column(data):

    # add columns to last and pop
    data['County'] = 'Sonoma'
    first_column = data.pop('County')
    data['Grape Color'] = 'Red'
    third_column = data.pop('Grape Color')

    # put county as first column, grape color as 3rd
    data.insert(0, 'County', first_column)
    data.insert(2, 'Grape Color', third_column)
    return data

def fix_incorrect_digits(data, year):
    # if exist a string that has empty space, add a '.' to complete the number
    
    if year == 2016:
        data['Bearing'] = data['Bearing'].str.replace(' .', '.', regex=False)
        data['Bearing'] = data['Bearing'].str.replace('--', '0.0', regex=False)
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' .', '.', regex=False)
        data['Non-Bearing'] = data['Non-Bearing'].str.replace('--', '0.0', regex=False)

        data['Total'] = data['Total'].str.replace(' .', '.', regex=False)
        data['Total'] = data['Total'].str.replace('--', '0.0', regex=False)
        data['Tons'] = data['Tons'].str.replace(' ', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' .', '.', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(' ', ',')
    elif year == 2015:
        data['$/Ton'] = data['$/Ton'].replace('$2.525.16', '$2,525.16')
        data['$/Ton'] = data['$/Ton'].replace('$3,525,42', '$3,525.42')
        data['Bearing'] = data['Bearing'].replace('11 689.7', '11,689.7')
    elif year == 2011:
        # for 2011 only 
        data['Bearing'] = data['Bearing'].str.replace(' ', '.')
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
        data['Total'] = data['Total'].str.replace(' ', '.')
        data['Tons'] = data['Tons'].str.replace(' ', '.')
        data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
        # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

        # specific for 2011
        data['Bearing'] = data['Bearing'].str.replace('*', '', regex=False)
        data['Tons'] = data['Tons'].str.replace('*', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace('*', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('\n', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('(a)  $ ', '$', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('*', '', regex=False)  
    elif year <= 2010:
        data['Bearing'] = data['Bearing'].str.replace(' ', '.')
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
        data['Total'] = data['Total'].str.replace(' ', '.')

        # # specific for 2010, 2009, 2008
        data['Tons'] = data['Tons'].str.replace('\n(a', '', regex=False)
        
        data['$/Ton'] = data['$/Ton'].str.replace(') ', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' (a', '', regex=False)
        data['$/Ton']  = data['$/Ton'].replace('', '$0.0', regex=False)
        
        # specific for 2007
        data['$/Ton'] = data['$/Ton'].str.replace(' (a)', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(')', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(') ', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('5,240.200', '5,240,200', regex=False)

        # specific for 2006:
        data['Bearing'] = data['Bearing'].str.replace('*', '', regex=False)
        data['Total'] = data['Total'].str.replace('*', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' (a)', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(' \n', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(' ', '', regex=False)

        # specific for 2004:
        data['$/Ton'] = data['$/Ton'].str.replace(' \n', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' ', '', regex=False)
    
        data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)   

        # specific for 2003 and <= 2002 
        if year == 2003:
            data['Bearing'] = data['Bearing'] + '.0'
            data['Non-Bearing'] = data['Non-Bearing'] + '.0'
            data['Total'] = data['Total'] + '.0'
            data['Tons'] = data['Tons'].str.rstrip('0').replace('19,571.', '19,571.0')  
            
        elif year <= 2002:
            data['Bearing'] = data['Bearing'] + '.0'
            data['Non-Bearing'] = data['Non-Bearing'] + '.0'
            data['Total'] = data['Total'] + '.0'
            data['Tons'] = data['Tons'] + '.0'

    else:
        data['Bearing'] = data['Bearing'].str.replace(' ', '.')
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
        data['Total'] = data['Total'].str.replace(' ', '.')
        data['Tons'] = data['Tons'].str.replace(' ', '.')
        data['$/Ton'] = data['$/Ton'].str.replace(' ', '.')
        data['Total Value'] = data['Total Value'].str.replace(' ', ',')
    return data
        

In [ ]:
# for <= 2016 we need to do it separately because of formatting 

combined_data_up_to_2017 = pd.DataFrame()
combined_data_up_to_2001 = pd.DataFrame()

# this goes up to 2017 files
for table, year in zip(table_list, sonoma_data['Year']):
    # 2021 - 2017 has a pattern where first 3 rows didn't matter
    
    if year >= 2017:
        table.drop([0,1,2], axis=0, inplace=True)

        # rename column names
        data_varietal_replaced = rename_replace_varietal_column(table, year)
        data_shifted = add_shift_county_color_column(data_varietal_replaced)

        # if exist a string that has empty space, add a '.' to complete the number
        data = fix_incorrect_digits(data_shifted, year)

        print(f'number of row for year {year} table is: {len(data)}')

        combined_data_up_to_2017 = pd.concat([combined_data_up_to_2017, data])

        # display(data)
        # display(combined_data_up_to_2017)
        
    # skips 2012 and 2013, their PDFs are not extractable
    elif year <= 2016 and year != 2013 and year != 2012:
        if year == 2009 or year == 2007 or year == 2003 or year == 2002 or year == 2001:
            table.drop([0,1,2], axis=0, inplace=True)
        else:
            table.drop([0,1], axis=0, inplace=True)
            
        # rename column names
        data_varietal_replaced = rename_replace_varietal_column(table, year)
        data_shifted = add_shift_county_color_column(data_varietal_replaced)

        # if exist a string that has empty space, add a '.' to complete the number
        data = fix_incorrect_digits(data_shifted, year)

        print(f'number of row for year {year} table is: {len(data)}')
        display(data)

        combined_data_up_to_2001 = pd.concat([combined_data_up_to_2001, data])

In [ ]:
sonoma_2017 = combined_data_up_to_2017.drop_duplicates().reset_index(drop=True)
sonoma_2001 = combined_data_up_to_2001.drop_duplicates().reset_index(drop=True)
sonoma_2001

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
sonoma_2001_sorted = sonoma_2001.sort_values(['Year', 'Varietal'])
sonoma_2001_sorted = sonoma_2001_sorted.reset_index(drop=True)
sonoma_2001_sorted

In [ ]:
sonoma_2001_sorted_copy = sonoma_2001_sorted.copy()
sonoma_2001_sorted_copy

In [ ]:
red_2015 = sonoma_2001_sorted_copy.query('Year == "2015"')
red_2015[red_2015.duplicated(subset='Varietal', keep=False)]

In [ ]:
red_2015_updated = red_2015.drop_duplicates(subset=['Varietal'], keep='first')
red_2015_updated

In [ ]:
sonoma_2001_fixed2015 = pd.concat([sonoma_2001_sorted_copy.query('Year != "2015"'), red_2015_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2015_copy = sonoma_2001_fixed2015.copy()
sonoma_2001_fixed2015_copy

In [ ]:
sonoma_2001_fixed2015_copy.query('Year == "2015"')

In [ ]:
pd.set_option('display.max_rows', 300)

In [ ]:
sonoma_2001_fixed2015_copy.query('Year == "2014"')

In [ ]:
red_2014 = sonoma_2001_fixed2015_copy.query('Year == "2014"')
red_2014[red_2014.duplicated(subset='Varietal', keep=False)]

In [ ]:
red_2014_updated = red_2014.drop_duplicates(subset=['Varietal'], keep='first')
red_2014_updated

In [ ]:
sonoma_2001_fixed2014 = pd.concat([sonoma_2001_sorted_copy.query('Year != "2014"'), red_2014_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2014_copy = sonoma_2001_fixed2014.copy()
sonoma_2001_fixed2014_copy

## Excel Read in for 2012 and 2013; big dataframe stopped here

In [ ]:
df_2012_2013 = pd.read_excel('2013_2012_red_variety.xlsx', sheet_name=['2013', '2012'])
df_2012_2013

In [ ]:
df_2013 = df_2012_2013.get('2013')
df_2013_7_columns = df_2013.iloc[:, :7]
df_2013_last_column = df_2013.iloc[:, 8:9]
# df_2013_correct_columns.columns = ['Varietal, Year, Bearing, Non-Bearing, Total, Tons, $/Ton, Total Value']
df_2013_right_columns = pd.concat([df_2013_7_columns, df_2013_last_column], axis=1)
df_2013_right_columns.columns = ['Varietal', 'Year', 'Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
df_2013_right_columns

In [ ]:
df_2013_right_columns_droppedna = df_2013_right_columns.dropna(how='all')
df_2013_right_columns_droppedna

In [ ]:
varietal_names = np.array(['Cabernet Franc', 
                            'Cabernet Sauvignon', 
                            'Carignane', 
                            'Malbec', 
                            'Merlot', 
                            'Meunier', 
                            'Petite Sirah', 
                            'Petite Verdot', 
                            'Pinot Noir', 
                            'Sangiovese', 
                            'Syrah-Shiraz', 
                            'Zinfandel',
                            'TOTAL ALL REDS Includes other reds'])
varietal_names_repeated = varietal_names.repeat(2)
varietal_names_repeated

In [ ]:
df_2013_right_columns_droppedna['Varietal'] = varietal_names_repeated
df_2013_right_columns_droppedna['Year'] = df_2013_right_columns_droppedna['Year'].astype('int')
df_2013_right_columns_cleaned = df_2013_right_columns_droppedna.astype(str)
df_2013_right_columns_cleaned

In [ ]:
df_2012= df_2012_2013.get('2012')
df_2012 = df_2012.dropna(axis=1, how='all')
df_2012_right_columns = df_2012.iloc[:, :8]
df_2012_right_columns.columns = ['Varietal', 'Year', 'Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
df_2012_right_columns['Varietal'] = np.nan
df_2012_right_columns = df_2012_right_columns.dropna(how='all')
df_2012_right_columns

In [ ]:
varietal_names = np.array(['Cabernet Franc', 
                            'Cabernet Sauvignon', 
                            'Carignane', 
                            'Malbec', 
                            'Merlot', 
                            'Meunier', 
                            'Petite Sirah', 
                            'Petite Verdot', 
                            'Pinot Noir', 
                            'Sangiovese', 
                            'Syrah-Shiraz', 
                            'Zinfandel',
                            'TOTAL ALL REDS Includes other reds'])
varietal_names_repeated = varietal_names.repeat(2)
varietal_names_repeated

In [ ]:
df_2012_right_columns['Varietal'] = varietal_names_repeated
df_2012_right_columns['Year'] = df_2012_right_columns['Year'].astype('int')
df_2012_right_columns_cleaned = df_2012_right_columns.astype(str)
df_2012_right_columns_cleaned['Total Value'] = df_2012_right_columns_cleaned['Total Value'].str.replace('*', '', regex=False)
df_2012_right_columns_cleaned['Total Value'] = df_2012_right_columns_cleaned['Total Value'].str.replace(' ', '')
df_2012_right_columns_cleaned['$/Ton'] = df_2012_right_columns_cleaned['$/Ton'].str.replace(' ', '')
df_2012_right_columns_cleaned

## Show 2012 duplicates with matching versions

In [ ]:
report_11_12_13_combined = pd.concat([df_2013_right_columns_cleaned, df_2012_right_columns_cleaned])
report_11_12_13_combined_changed = report_11_12_13_combined.drop_duplicates().sort_values(['Varietal', 'Year']).reset_index(drop=True)
report_11_12_13_combined_changed[report_11_12_13_combined_changed.duplicated(subset=['Varietal','Year'], keep=False)]

## Drop 2012 duplicates; compare between 2013's 2012 version and 2012 version

In [ ]:
red_11_12_13_drop_duplicates = report_11_12_13_combined_changed.drop_duplicates(subset=['Varietal', 'Year'], keep='first')
red_11_12_13_drop_duplicates

In [ ]:
red_11_12_13_drop_duplicates['County'] = 'Sonoma'
red_11_12_13_drop_duplicates['Grape Color'] = 'Red'
county_column = red_11_12_13_drop_duplicates['County']
grape_color_column = red_11_12_13_drop_duplicates['Grape Color']
red_11_12_13_drop_duplicates = red_11_12_13_drop_duplicates.drop(['County', 'Grape Color'], axis=1)
red_11_12_13_drop_duplicates.insert(0, 'County', county_column)
red_11_12_13_drop_duplicates.insert(2, 'Grape Color', grape_color_column)
red_11_12_13_drop_duplicates

### Now because only 2012 is matched with 2013's 2012 versiona and 2012 version; we can only revised 2012 from the big dataframe

In [ ]:
sonoma_2012 = red_11_12_13_drop_duplicates.query('Year == "2012"')
sonoma_2012

In [ ]:
sonoma_2001_fixed2014_added2012 = pd.concat([sonoma_2001_fixed2014, sonoma_2012]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2014_added2012

## Matching 2011 and 2013

In [ ]:
sonoma_2011 = red_11_12_13_drop_duplicates.query('Year == "2011"')
sonoma_2013 = red_11_12_13_drop_duplicates.query('Year == "2013"')

In [ ]:
sonoma_2011 = sonoma_2011.replace('TOTAL ALL REDS Includes other reds', 'TOTAL ALL REDS Includes Other Reds')
sonoma_2011

In [ ]:
sonoma_2013 = sonoma_2013.replace('TOTAL ALL REDS Includes other reds', 'TOTAL ALL REDS Includes Other Reds')
sonoma_2013

In [ ]:
sonoma_2001_fixed2014_added2012.query('Year == "2011"')

In [ ]:
sonoma_2011_matched = pd.concat([sonoma_2001_fixed2014_added2012.query('Year == "2011"'), sonoma_2011]).sort_values('Varietal').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2011_matched

In [ ]:
sonoma_2001_fixed2014_added2011_12 = pd.concat([sonoma_2001_fixed2014_added2012.query('Year != "2011"'), sonoma_2011]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2014_added2011_12

In [ ]:
sonoma_2013_matched = pd.concat([sonoma_2001_fixed2014_added2011_12.query('Year == "2013"'), sonoma_2013]).sort_values('Varietal').drop_duplicates()
sonoma_2013_matched

In [ ]:
sonoma_2001_fixed11_16 = pd.concat([sonoma_2001_fixed2014_added2011_12.query('Year != "2013"'), sonoma_2013_matched]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16

In [ ]:
sonoma_2001_fixed11_16['Varietal'].unique()

In [ ]:
sonoma_2001_fixed11_16['Varietal'] = sonoma_2001_fixed11_16['Varietal'].replace('TOTAL ALL REDS Includes other reds', 'TOTAL ALL REDS Includes Other Reds')
sonoma_2001_fixed11_16['Varietal'] = sonoma_2001_fixed11_16['Varietal'].replace('TOTAL ALL REDS NOT including Other Reds', 'TOTAL ALL REDS NOT Including Other Reds')
sonoma_2001_fixed11_16['Varietal'].unique()

In [ ]:
sonoma_2001_fixed11_16

## Still need to fix 2010, 2009, 2008, 2007 (need to combine)

### 2006 discrepancy

In [ ]:
red_2006 = sonoma_2001_fixed11_16.query('Year == "2006"').sort_values('Varietal')


red_2006[red_2006.duplicated(subset='Varietal', keep=False)]

In [ ]:
red_2006_updated = red_2006.drop_duplicates(subset=['Varietal'], keep='first')
red_2006_updated

## Fixed 2006 discrepancy

In [ ]:
sonoma_2001_fixed11_16_06 = pd.concat([sonoma_2001_fixed11_16.query('Year != "2006"'), red_2006_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06

In [ ]:
red_2005_updated = sonoma_2001_fixed11_16_06.query('Year == "2005"').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2001_fixed11_16_06_05 = pd.concat([sonoma_2001_fixed11_16_06.query('Year != "2005"'), red_2005_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05

## Skipped 2004 because there was no problem found 

In [ ]:
sonoma_2001_fixed11_16_06_05_04 = sonoma_2001_fixed11_16_06_05


red_2003_updated = sonoma_2001_fixed11_16_06_05_04.query('Year == "2003"').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2001_fixed11_16_06_05_04_03 = pd.concat([sonoma_2001_fixed11_16_06_05_04.query('Year != "2003"'), red_2003_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03

In [ ]:
red_2002_updated = sonoma_2001_fixed11_16_06_05_04_03.query('Year == "2002"').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2001_fixed11_16_06_05_04_03_02 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03.query('Year != "2002"'), red_2002_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01 = sonoma_2001_fixed11_16_06_05_04_03_02
sonoma_2001_fixed11_16_06_05_04_03_02_01

## Read in 2000 crop report

In [ ]:
df_2000_1999 = pd.read_excel('2020 crop report.xlsx')
df_2000_1999 = df_2000_1999.drop(df_2000_1999.index[-2:])
df_2000_1999

In [ ]:
correct_col_1999_2000 = df_2000_1999.iloc[:, :10].dropna(axis=1, how='all')
correct_col_1999_2000.insert(1, 'Year', [2000, 1999] * (len(correct_col_1999_2000) // 2) + [2000] * (len(correct_col_1999_2000) % 2))
correct_col_1999_2000.columns = ['Varietal', 'Year', 'Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
correct_col_1999_2000
varietal_names = np.array(['Cabernet Franc', 
                            'Cabernet Sauvignon', 
                            'Carignane', 
                            'Merlot', 
                            'Meunier', 
                            'Napa Gamay',
                            'Petite Sirah', 
                            'Petite Verdot', 
                            'Pinot Noir', 
                            'Sangiovese', 
                            'Syrah-Shiraz', 
                            'Zinfandel',
                            'Other Reds',
                            'TOTAL ALL REDS NOT Including Other Reds'])
varietal_names_repeated = varietal_names.repeat(2)
varietal_names_repeated
correct_col_1999_2000['Varietal'] = varietal_names_repeated
correct_col_1999_2000

In [ ]:
correct_col_1999_2000 = correct_col_1999_2000.fillna('1')
correct_col_1999_2000['Bearing'] = correct_col_1999_2000['Bearing'] + '.0'
correct_col_1999_2000['Non-Bearing'] = correct_col_1999_2000['Non-Bearing'] + '.0'
correct_col_1999_2000['Total'] = correct_col_1999_2000['Total'] + '.0'
correct_col_1999_2000['Tons'] = correct_col_1999_2000['Tons'] + '.0'
correct_col_1999_2000['$/Ton'] = '$'+ correct_col_1999_2000['$/Ton'].str.replace(' ', '').str.replace('[$S]', '', regex=True) + '.0'
correct_col_1999_2000['Total Value'] = '$'+ correct_col_1999_2000['Total Value'].str.replace(' ', '').str.replace('[$S]', '', regex=True)
correct_col_1999_2000 = correct_col_1999_2000.astype(str)
correct_col_1999_2000

In [ ]:
red_2000 = correct_col_1999_2000.query('Year == "2000"').reset_index(drop=True)
red_2000.insert(0, 'County', 'Sonoma')
red_2000.insert(2, 'Grape Color', 'Red')
red_2000

In [ ]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01, red_2000]).sort_values(['Year', 'Varietal']).drop_duplicates().reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching

In [ ]:
red_2000_matched = sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching.query('Year == "2000"').drop_duplicates(subset=['Varietal'], keep='first')
red_2000_matched

In [ ]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching.query('Year != "2000"'), red_2000_matched]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00

## Fixing 2010 Crop Report - we decide to use 2011's 2010 version
### 1. Assumed 2011's TOTAL REDS Row contains TOTAL REDS Including Other Reds 


In [ ]:
red_2010 = sonoma_2001_fixed11_16_06_05_04_03_02_01_00.query('Year == "2010"')
red_2010_fixed = red_2010.query('Varietal != "Other Reds" & Varietal != "TOTAL ALL REDS NOT Including Other Reds"')
red_2010_fixed

In [ ]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01_00.query('Year != "2010"'), red_2010_fixed]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10

In [ ]:
red_2008_updated = sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10.query('Year == "2008"').drop_duplicates(subset=['Varietal'], keep='last')
red_2008_updated

In [ ]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10_08 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10.query('Year != "2008"'), red_2008_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10_08

In [ ]:
pd.set_option('display.max_rows', 300)

In [110]:
numeric_columns = ['Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
year_numeric = ['Year']
df = sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10_08
df[year_numeric] = df[year_numeric].apply(pd.to_numeric)
df_up_to_2009 = df.query('Year <= 2009')
df_up_to_2009[numeric_columns] = df_up_to_2009[numeric_columns].replace('[\$,]', '', regex=True)

# Fix 3453.0* error for 2005 

df_up_to_2009.query('Year == 2005')
df_up_to_2009['Non-Bearing'] = df_up_to_2009['Non-Bearing'].str.replace('*', '', regex=True)

df_up_to_2009[numeric_columns] = df_up_to_2009[numeric_columns].apply(pd.to_numeric)
df_up_to_2009


,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,2097.0,2226.53,4668600
1,Sonoma,Cabernet Sauvigno,Red,2000,7328.0,3688.0,11016.0,32772.0,2417.96,79240400
2,Sonoma,Cabernet Sauvignon,Red,2000,7328.0,3688.0,11016.0,32772.0,2418.00,79240400
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,1391.55,1065400
4,Sonoma,Merlot,Red,2000,6564.0,1058.0,7622.0,31480.0,2019.29,63566200
...,...,...,...,...,...,...,...,...,...,...
138,Sonoma,Pinot Noir,Red,2009,10746.2,1988.5,12734.7,31961.2,3043.08,97260500
139,Sonoma,Sangiovese,Red,2009,298.6,11.1,309.7,1210.6,2231.17,2701100
140,Sonoma,Syrah-Shiraz,Red,2009,1816.6,168.5,1985.1,5563.3,2314.65,12877100
141,Sonoma,TOTAL ALL REDS NOT Including Other Reds,Red,2009,37937.2,3636.2,41573.4,120717.0,2430.05,293348400


## Combining Other Reds +TOTAL ALL REDS NOT Including Other Reds = TOTAL ALL REDS Including Other Reds

In [129]:
pd.set_option('display.max_rows', 300)

In [135]:
grouped = df_up_to_2009.groupby('Year')

new_df = pd.DataFrame()

for year, group in grouped:
    # find the rows to combine 
    total_all_reds_row = group.query('Varietal == "TOTAL ALL REDS NOT Including Other Reds"')
    other_reds_row = group.query('Varietal == "Other Reds"')

    # print(total_all_reds_row)

    # Combine the rows
    combined_row = total_all_reds_row.copy()
    combined_row['Varietal'] = 'TOTAL ALL REDS Including Other Reds'
    combined_row[numeric_columns] += other_reds_row[numeric_columns].values

    # Append the combined row to the new dataframe
    # print(combined_row)
    new_df = pd.concat([new_df, combined_row])

df_up_to_2009_combined = pd.concat([df_up_to_2009, new_df])
df_up_to_2009_combined_updated = df_up_to_2009_combined.query('Varietal != "TOTAL ALL REDS NOT Including Other Reds" & Varietal != "Other Reds"')
df_up_to_2009_combined_updated = df_up_to_2009_combined_updated.sort_values(['Year', 'Varietal']).reset_index(drop=True)


In [136]:
df_up_to_2009_combined_updated

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,2097.0,2226.53,4668600.0
1,Sonoma,Cabernet Sauvigno,Red,2000,7328.0,3688.0,11016.0,32772.0,2417.96,79240400.0
2,Sonoma,Cabernet Sauvignon,Red,2000,7328.0,3688.0,11016.0,32772.0,2418.00,79240400.0
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,1391.55,1065400.0
4,Sonoma,Merlot,Red,2000,6564.0,1058.0,7622.0,31480.0,2019.29,63566200.0
5,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,2069.27,1318700.0
6,Sonoma,Napa Gamay,Red,2000,84.0,0.0,84.0,195.0,1143.58,222500.0
7,Sonoma,Petite Sirah,Red,2000,280.0,86.0,366.0,1073.0,2127.81,2284000.0
8,Sonoma,Petite Verdot,Red,2000,86.0,77.0,163.0,282.0,2176.29,614000.0
9,Sonoma,Pinot Noir,Red,2000,4973.0,3631.0,8604.0,17520.0,2094.39,36692700.0


# We dropped the rows, now is just turn int back to string, and apply $, ., etc

## Developing general function to add Other Reds + TOTAL ALL REDS NOT Including Other Reds

In [ ]:
red_2009 = sonoma_2001_fixed11_16.query('Year == "2009"').sort_values(['Varietal'])

rows_to_sum = red_2009[red_2009.index.isin([166, 172])]
rows_to_sum

In [ ]:
rows_to_sum.columns

In [ ]:
red_2009 = sonoma_2001_fixed11_16.query('Year == "2009"').sort_values(['Varietal'])

rows_to_sum = red_2009[red_2009.index.isin([166, 172])]
numeric_columns = ['Bearing', 'Non-Bearing',
       'Total', 'Tons', '$/Ton', 'Total Value']
# rows_to_sum[numeric_columns] = rows_to_sum[numeric_columns].replace(',', '', regex=True).replace('$', '', regex=True).apply(pd.to_numeric)
# rows_to_sum
rows_to_sum[numeric_columns] = rows_to_sum[numeric_columns].replace({'\$': '', ',': ''}, regex=True).apply(pd.to_numeric)
combined_row = pd.DataFrame(rows_to_sum.loc[[166, 172], numeric_columns].sum()).T
# combined_row['County'] = 'Sonoma'
# combined_row['Varietal'] = 'TOTAL ALL REDS including Other Reds'
# combined_row['Grape Color'] = 'Red'
# combined_row['Year'] = 2009
combined_row

In [ ]:
columns_to_move = ['County', 'Varietal', 'Grape Color', 'Year']
fixed_combine_row = pd.concat([combined_row[columns_to_move], combined_row.drop(columns=columns_to_move)], axis=1)
fixed_combine_row

In [ ]:
sonoma_2017_sorted = sonoma_2017.sort_values(['Year', 'Varietal'])
sonoma_2017_sorted

## Indivdual Year Testing

In [ ]:
#2011 is urls[10]
#2010 is urls[11]
#2009 is urls[12]
#2008 is urls[13]
#2007 is urls[14]
#2006 is urls[15]
#2005 is urls[16]
#2004 is urls[17]
#2003 is urls[18]
#2002 is urls[19]
#2001 is urls[20]
#2000 is urls[21]



urls[18]

In [ ]:
# utilize camelot to read pdf file at page 12, capture precisely with stream and close up the row gap with row_tol = 10

tables = camelot.read_pdf(urls[18], pages=page, flavor='stream', row_tol=8, split_text=True)
data = tables[0].df
dat = tables[0].parsing_report
dat

In [ ]:
camelot.plot(tables[0], kind='contour').show()

In [ ]:
data

In [ ]:
# data.columns = ['0', '1', '2', '3', '4', '5', '6', '7']
# data

In [ ]:
# drop rows that is not helpful
data = data.drop([0,1,2], axis=0)
data
# for 2011 report, also drop an empty row 

In [ ]:
# # For 2016 crop report:

# # temp is non-bearing
# data['Temp'] = data['3']  # Create a temporary column and store 'Bearing' values
# data['3'] = data['2']  # Move 'Non-Bearing' values to 'Bearing' column
# data['2'] = data['Temp']  # Move temporary values to 'Non-Bearing' column
# data.drop('Temp', axis=1, inplace=True)  # Drop the temporary column
# data

In [ ]:
# special for 2006, 2002; $ sign is in its separate column
# rename its column, add the dollar column with the value
# data.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


# data['7'] = data['6'] + data['7']
# data['9'] = data['8'] + data['9']
# data = data.drop(['6', '8'], axis=1)
# data = data[:28]
# data

In [ ]:
# special for 2010; $ sign is in its separate column
# rename its column, add the dollar column with the value
# data.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


# data['7'] = data['6'] + data['7']
# data['9'] = data['8'] + data['9']
# data = data.drop(['6', '8'], axis=1)
# data

In [ ]:
data = data[:28]
data.loc[28][0] = data.loc[29][0]
data.loc[29][0] = ''

In [ ]:
data

In [ ]:
# rename my columns
new_columns = ['Varietal', 'Year', 'Bearing' , 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
data = data.rename(columns=dict(zip(data.columns,new_columns)))
data

In [ ]:
if '.' in data['Tons'].str.rstrip('0'):
    data['Tons'].str.rstrip('0')

In [ ]:
trailing_zero_strip = data['Tons'].str.rstrip('0')
trailing_zero_strip.replace('19,571.', '19,571.0')

In [ ]:
trailing_zero_strip[20]

In [ ]:
# data['Bearing'] = data['Bearing'].str.replace(' .', '.', regex=False)
# data['Bearing'] = data['Bearing'].str.replace('--', '0.0', regex=False)
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' .', '.', regex=False)
# data['Non-Bearing'] = data['Non-Bearing'].str.replace('--', '0.0', regex=False)

# data['Total'] = data['Total'].str.replace(' .', '.', regex=False)
# data['Total'] = data['Total'].str.replace('--', '0.0', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace(' .', '.', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(' ', ',')

In [ ]:
# data['Total'] = pd.to_numeric(data['Total'], errors='coerce')
# data

In [ ]:
# data = data[:28]
# data

In [ ]:
# data.loc[28][0] = data.loc[29][0]
# data.loc[29][0] = ''

In [ ]:

# specific fixes for 2004

# 28		2010	37,804.4	1,606.3	39,410.7	108,481.0\n(a	) $2,255.85 (a	) $244,717,100
# 29	TOTAL REDS	2009	37,937.2	3,636.2	41,573.4	120,717.0\n(a	) $2,430.05 (a	) $293,348,400	
			
# data.loc[28][0] = data.loc[29][0]
# data.loc[29][0] = ''

# # data drop empty row
# data = data.query("Year != '' and Total != '' and Tons != ''")
# data

# data drop empty row
# data = data[:28]
# data

In [ ]:
# data['Varietal'].unique()

In [ ]:
# data.loc[data['Varietal'].str.contains('Cabernet'), 'Varietal'] = 'Cabernet Sauvignon'
# data.loc[data['Varietal'].str.contains('Sangiovese/'), 'Varietal'] = 'Sangiovese/Sangioveto'

In [ ]:
# data['Varietal'].str.contains('Sangiovese/')

In [ ]:
data['Varietal'] = data['Varietal'].replace('Mataro/', 'Mataro/Mouvedere')
data['Varietal']= data['Varietal'].replace('Cabernet', 'Cabernet Sauvignon')

# for 2016 only 
data['Varietal'] = data['Varietal'].replace('Sangiovese/', 'Sangiovese')
# for 2016 Syrah - Shiraz to Syrah-Shiraz ; 
data['Varietal'] = data['Varietal'].replace('Syrah - Shiraz', 'Syrah-Shiraz')

# for 2005 only
data['Varietal'] = data['Varietal'].replace('Cabernet Sauv.', 'Cabernet Sauvignon')

data['Varietal']= data['Varietal'].replace(['', 'including other reds', 'Mouvedere', 'Mourvedere (a)', 'Sauvignon', 'Sangioveto'], np.nan)
data = data.reset_index(drop=True)

data['Varietal'].unique()

In [ ]:
data['Varietal'] = data['Varietal'].dropna().repeat(2).reset_index(drop=True)
data['County'] = 'Sonoma'

In [ ]:
first_column = data.pop('County')
data['Grape Color'] = 'Red'
third_column = data.pop('Grape Color')
data.insert(0, 'County', first_column)
data.insert(2, 'Grape Color', third_column)
data

In [ ]:
data['Tons'] 
data['$/Ton']
data['Total Value']

In [ ]:
data['Bearing'] + '.0'

In [ ]:
# for 2004 only 

data['Bearing'] = data['Bearing'].str.replace(' ', '.')
data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
data['Total'] = data['Total'].str.replace(' ', '.')

# data['Tons'] = data['Tons'].str.replace(' ', '.')
# data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # specific for 2004

# we could do .apply lambda function for all the possible combinations so we don't have to add specificly for each year

data['$/Ton'] = data['$/Ton'].str.replace(' \n', '', regex=False)
data['$/Ton'] = data['$/Ton'].str.replace(' ', '', regex=False)
data['Total Value'] = data['Total Value'].str.replace('  \n', '', regex=False)
data['Total Value'] = data['Total Value'].str.replace(' ', '', regex=False)
data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)
data

In [ ]:
# # for 2007 only 

# data['Bearing'] = data['Bearing'].str.replace(' ', '.')
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
# data['Total'] = data['Total'].str.replace(' ', '.')

# # data['Tons'] = data['Tons'].str.replace(' ', '.')
# # data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # # specific for 2006


# # we could do .apply lambda function for all the possible combinations so we don't have to add specificly for each year

# data['Bearing'] = data['Bearing'].str.replace('*', '', regex=False)
# data['Total'] = data['Total'].str.replace('*', '', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace(' (a)', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(' \n', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(' ', '', regex=False)
# data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)
# data

In [ ]:
# # for 2010, 2009, 2008 only 

# data['Bearing'] = data['Bearing'].str.replace(' ', '.')
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
# data['Total'] = data['Total'].str.replace(' ', '.')

# # data['Tons'] = data['Tons'].str.replace(' ', '.')
# # data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # # specific for 2010, 2009, 2008
# data['Tons'] = data['Tons'].str.replace('\n(a', '', regex=False)
# # data['$/Ton'] = data['$/Ton'].str.replace('', '0.0', regex=False)

# data['$/Ton'] = data['$/Ton'].str.replace(') ', '', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace(' (a', '', regex=False)
# data['$/Ton']  = data['$/Ton'].replace('', '$0.0', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(') ', '', regex=False)

# data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)
# data

In [ ]:
# data['$/Ton'].replace('', '$0.0', regex=False)

In [ ]:
# for 2011 only 

# data['Bearing'] = data['Bearing'].str.replace(' ', '.')
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
# data['Total'] = data['Total'].str.replace(' ', '.')
# data['Tons'] = data['Tons'].str.replace(' ', '.')
# data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # specific for 2011
# data['Tons'] = data['Tons'].str.replace('*', '', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace('*', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace('\n', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace('(a)  $ ', '$', regex=False)
# data['Total Value'] = data['Total Value'].str.replace('*', '', regex=False)

In [ ]:
data

In [ ]:
# for 2016 only 

# data['Bearing'].str.replace(' .', '.', regex=False)
# data['Bearing'].str.replace('--', '0.0', regex=False)
# data['Non-Bearing'].str.replace(' .', '.', regex=False)
# data['Non-Bearing'].str.replace('--', '0.0', regex=False)

# data['Total'].str.replace(' .', '.', regex=False)
# data['Total'].str.replace('--', '0.0', regex=False)
# data['$/Ton'].str.replace(' .', '.', regex=False)
# data['Total Value'].str.replace(' ', ',')

In [ ]:
len(data)

In [ ]:
data.to_csv('example.csv', index=False, encoding='utf-8')

In [ ]:
# data['Varietal'] = data['Varietal'].replace('', method='ffill')
# data